In [ ]:
from bastionlab import Identity

# Create `Identity` for Data owner.
data_owner = Identity.create("data_owner")

In [ ]:
!wget 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'

In [ ]:
!python datagen.py

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
!unzip smsspamcollection.zip

In [ ]:
!pip install transformers==0.13.2

In [2]:
from tokenizers import Tokenizer
from bastionlab.polars.policy import Policy, Aggregation, Log
from bastionlab.polars import train_test_split
import polars as pl
from bastionlab import Connection

file_path = "./SMSSpamCollection"

labels = []
texts = []
with open(file_path) as f:
    for line in f.readlines():
        split = line.split("\t")
        labels.append(1 if split[0] == "spam" else 0)
        texts.append(split[1])
df = pl.DataFrame({"label": labels, "text": texts})

connection = Connection("localhost")


policy = Policy(safe_zone=Aggregation(min_agg_size=10), unsafe_handling=Log())
rdf = connection.client.polars.send_df(df, policy=policy, sanitized_columns=["text"])

ratio = 0.25
train_rdf, test_rdf = train_test_split(rdf, test_size=ratio, shuffle=True)

tokenizer = Tokenizer.from_pretrained("distilbert-base-uncased")
tokenizer.enable_truncation(max_length=32)
tokenizer.enable_padding(length=32)

train_rds = (
    train_rdf.convert(["text"], tokenizer.to_str())
    .collect()
    .to_dataset(["text_ids", "text_mask"], "label")
)
test_rds = (
    test_rdf.convert(["text"], tokenizer.to_str())
    .collect()
    .to_dataset(["text_ids", "text_mask"], "label")
)


train_rds._set_test_dataset(test_rds.train_dataset_ref)
test_rds.set_train_dataset(train_dataset=train_rds.train_dataset_ref)

ValueError: At least one array required as input

In [ ]:
! pip install transformers

In [2]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
from bastionlab.torch.utils import MultipleOutputWrapper

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False,
    torchscript=True,
)
model = MultipleOutputWrapper(model, 0)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

In [3]:
remote_datasets = connection.client.torch.list_remote_datasets()
print(remote_datasets)

remote_datasets[0].trace_input

[tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0])]

In [4]:
from bastionlab.torch.optimizer_config import Adam

remote_learner = connection.client.torch.RemoteLearner(
    model,
    remote_datasets[0],
    max_batch_size=2,
    loss="cross_entropy",
    optimizer=Adam(lr=5e-5),
    model_name="DistilBERT",
    expand=False,
)

Sending DistilBERT: 100%|████████████████████| 268M/268M [00:06<00:00, 39.5MB/s] 


In [5]:
remote_learner.fit(nb_epochs=2, eps=None)

Epoch 1/2 - train:   1%|                    | 12/2090 [00:23<39:52,  1.15s/batch, cross_entropy=0.0000 (+/- 1553.7606)]  

KeyboardInterrupt: 